Importing Files


In [28]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC

Reading data

In [29]:
training_df=pd.read_csv("/content/training.csv") # Reading the training data from a CSV file into a pandas DataFrame
testing_df=pd.read_csv("/content/public_test_features.csv")# Reading the testing data (public test features) from a CSV file into a pandas DataFrame

<ipython-input-29-dccce5d7c693>:1: DtypeWarning: Columns (26,34,41,139,154,157,212) have mixed types. Specify dtype option on import or set low_memory=False.
  training_df=pd.read_csv("/content/training.csv")
<ipython-input-29-dccce5d7c693>:2: DtypeWarning: Columns (134,138,156,197,211) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_df=pd.read_csv("/content/public_test_features.csv")


In [30]:
training_df.head() # Displaying the first few rows of the training DataFrame to inspect the data

,ID,label,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,34016,0,1,1,B00YCZ6IKA,kitchen,Kitchen,NICLW,NaN,base_product,...,0.529104,pounds,5.118110,NaN,18-Apr-13,14-Oct-17,N,13-Oct-17,2867.0,NaN
1,3581,0,1,1,B00U25WT7A,office_product,Office Product,NaN,NaN,base_product,...,0.100000,pounds,4.500000,NaN,19-May-16,21-Mar-18,N,20-Mar-18,65.0,NaN
2,36025,1,1,1,B011BZ3GXU,consumer_electronics,Electronics,NaN,NaN,base_product,...,0.654773,pounds,3.937008,NaN,10-Dec-15,16-Feb-18,N,15-Feb-18,1532.0,NaN
3,42061,1,1,1,B0089XDG3I,pc,Personal Computers,NaN,NaN,base_product,...,3.549442,pounds,10.314961,NaN,19-Oct-12,15-Feb-18,N,14-Feb-18,13964.0,NaN
4,14628,1,1,1,B014UTSBZW,miscellaneous,Misc.,ZUKC7,1.0,base_product,...,0.396832,pounds,5.196850,NaN,26-Jul-12,9-Mar-18,N,9-Mar-18,1253.0,NaN


Checking Null Values

In [31]:
training_df.isnull() # Checking for missing values (null values) in the training DataFrame

,ID,label,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,False,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,True
1,False,False,False,False,False,False,False,True,True,False,...,False,False,False,True,False,False,False,False,False,True
2,False,False,False,False,False,False,False,True,True,False,...,False,False,False,True,False,False,False,False,False,True
3,False,False,False,False,False,False,False,True,True,False,...,False,False,False,True,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30007,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
30008,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
30009,False,False,False,False,False,True,True,True,True,False,...,False,False,False,True,False,False,False,False,False,True
30010,False,False,False,False,False,False,False,True,True,False,...,False,False,False,True,False,False,False,False,False,True


In [32]:
training_df.isnull().sum() # Counting the number of missing values in each column of the training DataFrame

ID                                         0
label                                      0
key_Region Id                              0
key_MarketPlace Id                         0
key_ASIN                                   0
                                       ...  
cand_dw_last_updated                       0
cand_is_deleted                            0
cand_last_updated                          0
cand_version                               1
cand_external_testing_certification    29546
Length: 228, dtype: int64

Preprocessing

In [33]:
# Removing duplicate rows from the training DataFrame and updating the DataFrame with the modified data
training_df=training_df.drop_duplicates()
training_df

,ID,label,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,34016,0,1,1,B00YCZ6IKA,kitchen,Kitchen,NICLW,NaN,base_product,...,0.529104,pounds,5.118110,NaN,18-Apr-13,14-Oct-17,N,13-Oct-17,2867.0,NaN
1,3581,0,1,1,B00U25WT7A,office_product,Office Product,NaN,NaN,base_product,...,0.100000,pounds,4.500000,NaN,19-May-16,21-Mar-18,N,20-Mar-18,65.0,NaN
2,36025,1,1,1,B011BZ3GXU,consumer_electronics,Electronics,NaN,NaN,base_product,...,0.654773,pounds,3.937008,NaN,10-Dec-15,16-Feb-18,N,15-Feb-18,1532.0,NaN
3,42061,1,1,1,B0089XDG3I,pc,Personal Computers,NaN,NaN,base_product,...,3.549442,pounds,10.314961,NaN,19-Oct-12,15-Feb-18,N,14-Feb-18,13964.0,NaN
4,14628,1,1,1,B014UTSBZW,miscellaneous,Misc.,ZUKC7,1.0,base_product,...,0.396832,pounds,5.196850,NaN,26-Jul-12,9-Mar-18,N,9-Mar-18,1253.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30007,23247,0,1,1,B00L7RU8C0,kitchen,Kitchen,NESD9,1.0,base_product,...,2.150000,pounds,7.600000,NaN,27-Dec-12,27-Oct-17,N,26-Oct-17,4476.0,NaN
30008,41752,1,1,1,B00IF70T4M,consumer_electronics,Electronics,1BYO0,1.0,base_product,...,0.050000,pounds,14.400000,NaN,31-Aug-15,3-Dec-16,N,3-Dec-16,51.0,NaN
30009,38909,1,1,1,B0148DZSYW,NaN,NaN,NaN,NaN,base_product,...,0.050706,pounds,3.188976,NaN,29-Nov-16,1-Feb-18,N,31-Jan-18,63.0,NaN
30010,25230,0,1,1,B00QSSHFCO,consumer_electronics,Electronics,NaN,NaN,base_product,...,0.132277,pounds,2.913386,NaN,28-Apr-16,7-Mar-18,N,6-Mar-18,321.0,NaN


In [34]:
# Finding columns with all missing values in the training DataFrame
empty_columns = training_df.columns[training_df.isnull().sum() == training_df.shape[0]]
# Dropping columns with all missing values from the training DataFrame
training_df = training_df.drop(empty_columns, axis=1)
# Printing the modified training DataFrame after dropping empty columns
print(training_df)

          ID  label  key_Region Id  key_MarketPlace Id    key_ASIN  \
0      34016      0              1                   1  B00YCZ6IKA   
1       3581      0              1                   1  B00U25WT7A   
2      36025      1              1                   1  B011BZ3GXU   
3      42061      1              1                   1  B0089XDG3I   
4      14628      1              1                   1  B014UTSBZW   
...      ...    ...            ...                 ...         ...   
30007  23247      0              1                   1  B00L7RU8C0   
30008  41752      1              1                   1  B00IF70T4M   
30009  38909      1              1                   1  B0148DZSYW   
30010  25230      0              1                   1  B00QSSHFCO   
30011  16210      1              1                   1  B003EM8008   

           key_Binding Code key_binding_description key_brand_code  \
0                   kitchen                 Kitchen          NICLW   
1            office

In [35]:
# Filling missing values (NaN) with an empty string ('') in the training DataFrame
training_df=training_df.fillna('')
training_df

,ID,label,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,34016,0,1,1,B00YCZ6IKA,kitchen,Kitchen,NICLW,,base_product,...,0.529104,pounds,5.11811,,18-Apr-13,14-Oct-17,N,13-Oct-17,2867.0,
1,3581,0,1,1,B00U25WT7A,office_product,Office Product,,,base_product,...,0.1,pounds,4.5,,19-May-16,21-Mar-18,N,20-Mar-18,65.0,
2,36025,1,1,1,B011BZ3GXU,consumer_electronics,Electronics,,,base_product,...,0.654773,pounds,3.937008,,10-Dec-15,16-Feb-18,N,15-Feb-18,1532.0,
3,42061,1,1,1,B0089XDG3I,pc,Personal Computers,,,base_product,...,3.549442,pounds,10.314961,,19-Oct-12,15-Feb-18,N,14-Feb-18,13964.0,
4,14628,1,1,1,B014UTSBZW,miscellaneous,Misc.,ZUKC7,1.0,base_product,...,0.396832,pounds,5.19685,,26-Jul-12,9-Mar-18,N,9-Mar-18,1253.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30007,23247,0,1,1,B00L7RU8C0,kitchen,Kitchen,NESD9,1.0,base_product,...,2.15,pounds,7.6,,27-Dec-12,27-Oct-17,N,26-Oct-17,4476.0,
30008,41752,1,1,1,B00IF70T4M,consumer_electronics,Electronics,1BYO0,1.0,base_product,...,0.05,pounds,14.4,,31-Aug-15,3-Dec-16,N,3-Dec-16,51.0,
30009,38909,1,1,1,B0148DZSYW,,,,,base_product,...,0.050706,pounds,3.188976,,29-Nov-16,1-Feb-18,N,31-Jan-18,63.0,
30010,25230,0,1,1,B00QSSHFCO,consumer_electronics,Electronics,,,base_product,...,0.132277,pounds,2.913386,,28-Apr-16,7-Mar-18,N,6-Mar-18,321.0,


In [36]:
"""
    Fill missing values in DataFrame based on column data type.

    Parameters:
    df (DataFrame): Input DataFrame containing missing values.

    Returns:
    DataFrame: DataFrame with missing values filled based on data types.
"""
def fill_blanks(df):
  for column in df:
    if df[column].dtype == np.int64:
      df[column] = df[column].fillna(0)
    elif df[column].dtype == np.float64:
      df[column] = df[column].fillna(0.0)
    else:
      df[column] = df[column].fillna('')
  return df

training_df = fill_blanks(training_df)
training_df


,ID,label,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,34016,0,1,1,B00YCZ6IKA,kitchen,Kitchen,NICLW,,base_product,...,0.529104,pounds,5.11811,,18-Apr-13,14-Oct-17,N,13-Oct-17,2867.0,
1,3581,0,1,1,B00U25WT7A,office_product,Office Product,,,base_product,...,0.1,pounds,4.5,,19-May-16,21-Mar-18,N,20-Mar-18,65.0,
2,36025,1,1,1,B011BZ3GXU,consumer_electronics,Electronics,,,base_product,...,0.654773,pounds,3.937008,,10-Dec-15,16-Feb-18,N,15-Feb-18,1532.0,
3,42061,1,1,1,B0089XDG3I,pc,Personal Computers,,,base_product,...,3.549442,pounds,10.314961,,19-Oct-12,15-Feb-18,N,14-Feb-18,13964.0,
4,14628,1,1,1,B014UTSBZW,miscellaneous,Misc.,ZUKC7,1.0,base_product,...,0.396832,pounds,5.19685,,26-Jul-12,9-Mar-18,N,9-Mar-18,1253.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30007,23247,0,1,1,B00L7RU8C0,kitchen,Kitchen,NESD9,1.0,base_product,...,2.15,pounds,7.6,,27-Dec-12,27-Oct-17,N,26-Oct-17,4476.0,
30008,41752,1,1,1,B00IF70T4M,consumer_electronics,Electronics,1BYO0,1.0,base_product,...,0.05,pounds,14.4,,31-Aug-15,3-Dec-16,N,3-Dec-16,51.0,
30009,38909,1,1,1,B0148DZSYW,,,,,base_product,...,0.050706,pounds,3.188976,,29-Nov-16,1-Feb-18,N,31-Jan-18,63.0,
30010,25230,0,1,1,B00QSSHFCO,consumer_electronics,Electronics,,,base_product,...,0.132277,pounds,2.913386,,28-Apr-16,7-Mar-18,N,6-Mar-18,321.0,


In [37]:
def fill_blanks_with_mean(df):
  """
    Fill missing values in DataFrame based on column data type using mean or mode.

    Parameters:
    df (DataFrame): Input DataFrame containing missing values.

    Returns:
    DataFrame: DataFrame with missing values filled based on mean or mode of each column.
    """
  for column in df:
    if df[column].dtype == np.int64:
      df[column] = df[column].fillna(df[column].mean())
    elif df[column].dtype == np.float64:
      df[column] = df[column].fillna(df[column].mean())
    else:
      df[column] = df[column].fillna(df[column].mode()[0])
  return df

training_df = fill_blanks_with_mean(training_df)
training_df


,ID,label,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,34016,0,1,1,B00YCZ6IKA,kitchen,Kitchen,NICLW,,base_product,...,0.529104,pounds,5.11811,,18-Apr-13,14-Oct-17,N,13-Oct-17,2867.0,
1,3581,0,1,1,B00U25WT7A,office_product,Office Product,,,base_product,...,0.1,pounds,4.5,,19-May-16,21-Mar-18,N,20-Mar-18,65.0,
2,36025,1,1,1,B011BZ3GXU,consumer_electronics,Electronics,,,base_product,...,0.654773,pounds,3.937008,,10-Dec-15,16-Feb-18,N,15-Feb-18,1532.0,
3,42061,1,1,1,B0089XDG3I,pc,Personal Computers,,,base_product,...,3.549442,pounds,10.314961,,19-Oct-12,15-Feb-18,N,14-Feb-18,13964.0,
4,14628,1,1,1,B014UTSBZW,miscellaneous,Misc.,ZUKC7,1.0,base_product,...,0.396832,pounds,5.19685,,26-Jul-12,9-Mar-18,N,9-Mar-18,1253.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30007,23247,0,1,1,B00L7RU8C0,kitchen,Kitchen,NESD9,1.0,base_product,...,2.15,pounds,7.6,,27-Dec-12,27-Oct-17,N,26-Oct-17,4476.0,
30008,41752,1,1,1,B00IF70T4M,consumer_electronics,Electronics,1BYO0,1.0,base_product,...,0.05,pounds,14.4,,31-Aug-15,3-Dec-16,N,3-Dec-16,51.0,
30009,38909,1,1,1,B0148DZSYW,,,,,base_product,...,0.050706,pounds,3.188976,,29-Nov-16,1-Feb-18,N,31-Jan-18,63.0,
30010,25230,0,1,1,B00QSSHFCO,consumer_electronics,Electronics,,,base_product,...,0.132277,pounds,2.913386,,28-Apr-16,7-Mar-18,N,6-Mar-18,321.0,


In [38]:
training_df.dropna(axis=1, how='any', inplace=True) # Drop columns containing any missing values from the training DataFrame

In [39]:
# Select columns with data type 'object' (varchar) from the training DataFrame
varchar_columns = training_df.select_dtypes(include=['object']).columns
# Iterate through varchar columns and convert them to numeric type, coercing errors to NaN
for col in varchar_columns:
    training_df[col] = pd.to_numeric(training_df[col], errors='coerce')

print(training_df)

          ID  label  key_Region Id  key_MarketPlace Id  key_ASIN  \
0      34016      0              1                   1       NaN   
1       3581      0              1                   1       NaN   
2      36025      1              1                   1       NaN   
3      42061      1              1                   1       NaN   
4      14628      1              1                   1       NaN   
...      ...    ...            ...                 ...       ...   
30007  23247      0              1                   1       NaN   
30008  41752      1              1                   1       NaN   
30009  38909      1              1                   1       NaN   
30010  25230      0              1                   1       NaN   
30011  16210      1              1                   1       NaN   

       key_Binding Code  key_binding_description  key_brand_code  \
0                   NaN                      NaN             NaN   
1                   NaN                      Na

In [40]:
# Creating the feature matrix X_train by dropping the 'ID' and 'label' columns from the training DataFrame
X_train = training_df.drop(['ID', 'label'], axis=1)
# Creating the target vector y_train containing the 'label' column values from the training DataFrame
y_train = training_df['label']

In [41]:
# Creating a SimpleImputer instance with strategy="mean" to impute missing values with column means
imputer = SimpleImputer(strategy="mean")
# Imputing missing values in the feature matrix X_train using the imputer
X_train = imputer.fit_transform(X_train)

In [42]:
X_train_df = pd.DataFrame(X_train)
# Creating a DataFrame X_train_df from the feature matrix X_train
label_encoders = {}   # Dictionary to store LabelEncoder instances for each categorical column
# Looping through columns of X_train_df with data types 'int' or 'string' (object)
for col in X_train_df.select_dtypes(include=['int','string']).columns:
    label_encoders[col] = LabelEncoder()
    X_train_df[col] = label_encoders[col].fit_transform(X_train_df[col])
# Looping through columns of X_train_df to convert 'int64' columns to numeric data type
for col in X_train_df.columns:
    if X_train_df[col].dtype == 'int64':
        X_train_df[col] = pd.to_numeric(X_train_df[col], errors='ignore')
# Converting X_train_df back to a NumPy array for further processing
X_train = X_train_df.values

In [43]:
print(y_train.head()) #print first few rows of y_train

0    0
1    0
2    1
3    1
4    1
Name: label, dtype: int64


Applying Random Forest

In [44]:
model = RandomForestClassifier() # Creating a RandomForestClassifier instance
model.fit(X_train, y_train) # Fitting the RandomForestClassifier model to the training data

RandomForestClassifier()

In [53]:
# Splitting the data into training and validation sets
X_train_eval, X_val, y_train_eval, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=45)
# Fitting the model to the training data
model.fit(X_train_eval, y_train_eval)
# Making predictions on the validation set
y_pred = model.predict(X_val)
# Calculating the accuracy of the model on the validation set
accuracy = accuracy_score(y_val, y_pred)
# Generating a classification report with detailed performance metrics on the validation set

report = classification_report(y_val, y_pred)


In [54]:
print(f'Accuracy: {accuracy}') # Print the accuracy of the model
print(f'Classification Report:\n{report}') # Print the classification report

Accuracy: 0.6636681659170415
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.67      0.67      2995
           1       0.67      0.66      0.66      3008

    accuracy                           0.66      6003
   macro avg       0.66      0.66      0.66      6003
weighted avg       0.66      0.66      0.66      6003



Applying Gradient Boosting

In [55]:
# Creating a GradientBoostingClassifier instance with specified parameters
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.2, random_state=42)
# Fitting the GradientBoostingClassifier model to the training data
gb_classifier.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.2, random_state=42)

In [56]:
# Making predictions on the training set using the GradientBoostingClassifier
y_pred_gb = gb_classifier.predict(X_train)
# Calculating the accuracy of the GradientBoostingClassifier on the training set
accuracy_gb = accuracy_score(y_train, y_pred_gb)
# Printing the accuracy of the GradientBoostingClassifier
print(f'Gradient Boosting Classifier Accuracy: {accuracy_gb}')
# Generating a classification report with detailed performance metrics on the training set
report=classification_report(y_train, y_pred_gb)
# Printing the classification report
print(f'Classification Report:\n{report}')

Gradient Boosting Classifier Accuracy: 0.6721311475409836
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.61      0.65     14840
           1       0.66      0.74      0.69     15172

    accuracy                           0.67     30012
   macro avg       0.67      0.67      0.67     30012
weighted avg       0.67      0.67      0.67     30012

